In [24]:
#! pip install google-cloud-dialogflow
#! pip install python-dotenv


### Choose GCP project ID

Move to GCP console, you can find project_id will be used in the DialogFlow agent.


![project list](resources/df_1.project_selection.png)

I stored project id into my environment variable - PROJECT_ID. 

In [25]:
import os
from dotenv import load_dotenv

# override env variable values.
# set the 'PROJECT_ID' values in the '/.env' file. (or set your project_id value directly)
load_dotenv() 

PROJECT_ID=os.getenv('PROJECT_ID')
AGENT_DISPLAY_NAME="dialogflow_with_apis"
LANG_CODE="ko"
TIME_ZONE="Asia/Seoul"



In [26]:
from google.cloud.dialogflow_v2 import Agent, AgentsClient, SetAgentRequest
import google.protobuf.field_mask_pb2

def set_agent(project_id, display_name, language_code, time_zone):
    agents_client = AgentsClient()

    parent = agents_client.common_project_path(project_id)

    agent = Agent(
        parent=parent,
        display_name=display_name,
        default_language_code=language_code, # "en",
        time_zone=time_zone #"America/Los_Angeles",
    )

    update_mask = google.protobuf.field_mask_pb2.FieldMask()
    update_mask.FromJsonString("displayName,defaultLanguageCode,timeZone")

    request = SetAgentRequest(
        agent=agent,
        update_mask=update_mask,
    )

    return agents_client.set_agent(request=request)


In [27]:
dialogflow_agent = set_agent(
    PROJECT_ID, 
    AGENT_DISPLAY_NAME,
    LANG_CODE,
    TIME_ZONE

)

### Verify the agent exists on the DialogFlow Console. 

After creation an agent with API, you can see the agent on the DialogFlow Console.

![dialog flow agent](resources/df_2.dialogflow_agent_creation.png)

### Create a new Intent

Intent 구성이 정적일 경우면, 즉 정확한 의도를 잘 찾고 해당하는 의도에 맞는 답변을 "정적"으로 돌려주는 것이라면, 
Intent 구성만 진행하면 된다. 

하지만 대부분의 경우, Response를 제공해 주기 위하여 Backend에 있는 REST API등을 호출하여 원하는 결과를 동적으로 수행하게 만들어 주는 경우가 많으며, 
이 경우, REST API호출을 담당하는 Fullfilment 구성을 수행하여야 한다. 

먼저, 인텐트 전체 목록을 가지고 오자.


In [28]:
def list_intents(project_id):
    from google.cloud import dialogflow

    intents_client = dialogflow.IntentsClient()

    parent = dialogflow.AgentsClient.agent_path(project_id)

    intents = intents_client.list_intents(request={"parent": parent})

    for intent in intents:
        print("=" * 20)
        #print("Intent name: {}".format(intent.name))
        print("Intent display_name: {}".format(intent.display_name))
        print("Action: {}\n".format(intent.action))
        print("Root followup intent: {}".format(intent.root_followup_intent_name))
        print("Parent followup intent: {}\n".format(intent.parent_followup_intent_name))

        print("Input contexts:")
        for input_context_name in intent.input_context_names:
            print("\tName: {}".format(input_context_name))

        print("Output contexts:")
        for output_context in intent.output_contexts:
            print("\tName: {}".format(output_context.name))



In [23]:
list_intents(PROJECT_ID)


Intent display_name: Default Fallback Intent
Action: input.unknown

Root followup intent: 
Parent followup intent: 

Input contexts:
Output contexts:
Intent display_name: Default Welcome Intent
Action: input.welcome

Root followup intent: 
Parent followup intent: 

Input contexts:
Output contexts:


In [ ]:

def create_intent(project_id, display_name, training_phrases_parts, message_texts):
    """Create an intent of the given intent type."""
    from google.cloud import dialogflow

    intents_client = dialogflow.IntentsClient()

    parent = dialogflow.AgentsClient.agent_path(project_id)
    training_phrases = []
    for training_phrases_part in training_phrases_parts:
        part = dialogflow.Intent.TrainingPhrase.Part(text=training_phrases_part)
        # Here we create a new training phrase for each provided part.
        training_phrase = dialogflow.Intent.TrainingPhrase(parts=[part])
        training_phrases.append(training_phrase)

    text = dialogflow.Intent.Message.Text(text=message_texts)
    message = dialogflow.Intent.Message(text=text)

    intent = dialogflow.Intent(
        display_name=display_name, training_phrases=training_phrases, messages=[message]
    )

    response = intents_client.create_intent(
        request={"parent": parent, "intent": intent}
    )

    print("Intent created: {}".format(response))



### Modify Fallback Intent. 

Fallback intent has no session information. 

It means if the dialog session would be needed, the session should be kept on the Fullfillment webhook side. 

폴백 인텐트는 기본적으로 매칭되는 인텐트가 없을 때, 대응하는 인텐트이다.

폴백 인텐트에 Search 기능을 담아보자.





# Create a function to create a webhook.
def create_webhook():
    data = {
        "parent": "projects/project-id/agent",
        "displayName": "webhook-display-name",
        "uri": "https://example.com/webhook"
    }
    response = send_request("POST", base_url + "projects/project-id/agent/webhooks", data=data)
    return response

# Create a webhook.
webhook = create_webhook()

# Create a function to detect intent.
def detect_intent():
    data = {
        "queryInput": {
            "text": {
                "text": "What is the weather today?"
            }
        },
        "queryParams": {
            "timeZone": "America/Los_Angeles"
        }
    }
    response = send_request("POST", base_url + "projects/project-id/agent/sessions/1234567890:detectIntent", data=data)
    return response

# Detect intent.
response = detect_intent()

# Print the response.
print(response)